In [6]:
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from llama_index.prompts.prompts import SimpleInputPrompt
import torch
import os

os.chdir(r"D:\Projects\Offline_translator-webapp")

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [17]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-AWQ"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

Another snippet

In [14]:
# System prompt (for ASR cleanup task)
system_prompt = """You are a language refinement assistant that receives raw text generated by a speech-to-text system.
This text may be in either Persian or English and may contain errors in spacing, punctuation, spelling, or transcription.
Your task is to clean the text and output a corrected version in the **same language as the input**.
If the user input is in Persian, the output should be in Persian. If the user input is in English, the output should be in English.
The output should be a single, coherent sentence or phrase without any additional formatting or tags.
Do not translate. Do not add any explanation, punctuation symbols, emojis, or formatting like '✅' — only return the cleaned-up text, nothing else."""


# Example input with ASR errors
# noisy_input = "پادشاهاز کوره دررفت و در درون چا افتاد"
noisy_input = "این حرف از دهن دختر اومد بیرون اونها ای که دوروبر پادشا ها جم شده بودن گفتن بهتره سر این جوانو زیر آب بکنیم و نظریم از این قصر پا بیرون بزاره و گنه آوازش می افت به دهن مردم آبرو شاه و دخترش می ره"

# Build ChatML-formatted prompt
prompt = f"""<|im_start|>system
{system_prompt}
<|im_end|>
<|im_start|>user
{noisy_input}
<|im_end|>
<|im_start|>assistant
"""

# Tokenize
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

# Generate response
with torch.no_grad():
    output = model.generate(
        input_ids,
        max_new_tokens=128,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id,
    )

# Extract model response
decoded = tokenizer.decode(output[0], skip_special_tokens=True)
response = decoded.split("<|im_start|>assistant")[-1].strip()
# response = response.split("<|im_start|>")[0].strip()

# Remove unwanted markers like ✅, 'Refined Output:', and chat tags
junk_prefixes = ["<|im_end|>"]
for junk in junk_prefixes:
    if junk in response:
        response = response.replace(junk, "").strip()

# Optionally: remove any remaining formatting or assistant continuation
response = response.split("<|im_start|>")[0].strip()

print(response)

این جوان دهن دختران را از پادشاهان بیرون کند، بهتره سر این یونان زیر اب برون کنیم و از این قصر پا بیرون بزاره گنه آوازش می افت به


In [9]:
response

'پاد'

PersianLLM : PersianMind

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    "universitytehran/PersianMind-v1.0",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    device_map={"": device},
)

tokenizer = AutoTokenizer.from_pretrained(
    "universitytehran/PersianMind-v1.0",
)

config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

D:\Projects\venv2\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User_1\.cache\huggingface\hub\models--universitytehran--PersianMind-v1.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
TEMPLATE = "{context}\nYou: {prompt}\nPersianMind: "

CONTEXT = "This is a conversation with PersianMind. It is an artificial intelligence model designed by a team of " \
    "NLP experts at the University of Tehran to help you with various tasks such as answering questions, " \
    "providing recommendations, and helping with decision making. You can ask it anything you want and " \
    "it will do its best to give you accurate and relevant information."
PROMPT = "در مورد هوش مصنوعی توضیح بده."

model_input = TEMPLATE.format(context=CONTEXT, prompt=PROMPT)
input_tokens = tokenizer(model_input, return_tensors="pt")
input_tokens = input_tokens.to(device)
generate_ids = model.generate(**input_tokens, max_new_tokens=512, do_sample=False, repetition_penalty=1.1)
model_output = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

print(model_output[len(model_input):])
